In [11]:
import sys # 시스템 : 쓸수도 있고 안쓸수도 있지만 일단 가져오기
import os  # 시스템 : 쓸수도 있고 안쓸수도 있지만 일단 가져오기

import pandas as pd  # 판다스 : 데이터분석 라이브러리
import numpy as np   # 넘파이 : 숫자, 행렬 데이터 라이브러리

from bs4 import BeautifulSoup    # html 데이터를 전처리 # 아직안씀
from selenium import webdriver   # 웹 브라우저 자동화
import time    # 서버와 통신할 때 중간중간 시간 지연. 보통은 1초
#from tqdm import tqdm_notebook   # tqdm이전 버전의 사용 방법. 버전이 완전히 바뀌면 아래 코드만을 사용해야 합니다.
from tqdm.notebook import tqdm    # for문 돌릴 때 진행상황을 %게이지로알려줍니다.

In [12]:
#Step 1. 크롬 웹브라우저 실행
path = "chromedriver.exe"       # 윈도우는 "chromedriver.exe"

driver = webdriver.Chrome(path)
# 사이트 주소는 melon(음원사이트)
driver.get('http://www.melon.com')
time.sleep(2)  # 2초간 정지

In [13]:
# 멜론차트 클릭
driver.find_element_by_css_selector(".menu_bg.menu01").click()
time.sleep(2)

# 일간차트 클릭
driver.find_element_by_css_selector(".menu_chart.m2").click()


# 크롤링 코드

dict = {}    # 전체 크롤링 데이터를 담을 그릇

number = 100  # 수집할 글 갯수 정하기

for i in range(0,number):
    # 곡정보 더보기 버튼 클릭
    more_info_list = driver.find_elements_by_css_selector(".btn.button_icons.type03.song_info")
    more_info_list[i].click()
    time.sleep(1)
    # 크롤링
    try : 
        music_info = {}  # 개별 블로그 내용을 담을 딕셔너리 생성
        time.sleep(1)
        
        # 제목 크롤링
        overlays = ".song_name"                                 
        tit = driver.find_element_by_css_selector(overlays)          # title
        title = tit.text

        # 가수 크롤링
        overlays = ".artist"                              
        nick = driver.find_element_by_css_selector(overlays)         # nickname
        artist_name = nick.text
        time.sleep(1)

        # 발매날짜 크롤링
        overlays = ".list"                                
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')         # datetime
        Album_date = meta_info[3]
        
        # 좋아요 수
        overlays = "d_like_count"
        like_count = driver.find_element_by_id(overlays)
        LIKE = like_count.text
        
        # 앨범이름 크롤링
        overlays = ".list"
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')
        Album_name = meta_info[1]
        
        # 현재 곡 장르 크롤링
        overlays = ".list"
        meta_info = driver.find_element_by_css_selector(overlays).text.split('\n')
        Album_Genre = meta_info[5]
        
        # 더보기 버튼 클릭
        driver.find_element_by_css_selector(".button_more.arrow_d").click()
        time.sleep(1)
        
        # 가사 크롤링
        overlays = ".lyric.on"                                 
        contents = driver.find_elements_by_css_selector(overlays)    # contents
        
        content_list = []
        for content in contents:
            content_list.append(content.text)
 
        content_str = ' '.join(content_list)                         # content_str

        # 글 하나는 target_info라는 딕셔너리에 담기게 되고,
        music_info['title'] = title
        music_info['artist'] = artist_name
        music_info['Album_date'] = Album_date
        music_info['LIKE'] = LIKE
        music_info['Album_name'] = Album_name
        music_info['Album_Genre'] = Album_Genre
        music_info['Lyric'] = content_str
        time.sleep(1)
        # 각각의 글은 dict라는 딕셔너리에 담기게 됩니다.
        dict[i] = music_info
        
        # 크롤링이 성공하면 글 제목을 출력하게 되고,
        print(i, title)

        # 글 하나 크롤링 후 뒤로가기
        driver.back()  
    
    # 에러처리 : 에러나면 뒤로가기하고 다음 글(i+1)로 이동합니다.
    except:
        driver.back()
        time.sleep(1)
        continue
    
    # 중간,중간에 파일로 저장하기  - 에러날 경우 데이터 손실 위험 감소
    if i == 30 or 50 or 80:
        # 판다스로 만들기
        import pandas as pd
        result_df = pd.DataFrame.from_dict(dict, 'index')

        # 저장하기
        result_df.to_excel("Music_melon.xlsx", encoding='utf-8-sig')
        time.sleep(2)

# 저장하기
result_df.to_excel("Music_melon.xlsx", encoding='utf-8-sig')
time.sleep(2)
driver.quit()

print('수집한 글 갯수: ', len(dict))

0 TOMBOY
1 GANADARA (Feat. 아이유)
2 Feel My Rhythm
3 사랑인가 봐
4 취중고백
5 INVU
6 듣고 싶을까
7 사랑은 늘 도망가
8 RUN2U
9 언제나 사랑해
10 다정히 내 이름을 부르면
11 SMILEY (Feat. BIBI)
12 신호등
13 ELEVEN
14 회전목마 (Feat. Zion.T, 원슈타인) (Prod. Slom)
15 STAY
16 드라마
17 존재만으로
18 Step Back
19 리무진 (Feat. MINO) (Prod. GRAY)
20 어제처럼
21 내 생에 아름다운
22 너, 너 (N번째 연애 X 휘인 (Whee In))
23 strawberry moon
24 버퍼링 (Glitch Mode)
25 Next Level
26 사랑인걸
28 호랑수월가
29 눈이 오잖아(Feat.헤이즈)
30 Weekend
31 Counting Stars (Feat. Beenzino)
32 라일락
33 너를 생각해
34 Dreams Come True
35 Dynamite
36 바라만 본다
37 OHAYO MY NIGHT
38 MY BAG
39 Savage
40 Starlight
41 Butter
42 만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. TOIL)
43 Celebrity
44 이제 나만 믿어요
45 Blueming
46 찰나가 영원이 될 때 (The Eternal Moment)
47 사이렌 Remix (Feat. UNEDUCATED KID, Paul Blanco)
48 Permission to Dance
49 내 손을 잡아
50 봄날
51 Bad Habits
52 겨울잠
53 밤하늘의 별을(2020)
54 서랍
55 문득
56 벚꽃 엔딩
57 헤픈 우연
58 With
59 모든 날, 모든 순간 (Every day, Every Moment)
60 고백
61 Can't Control Myself
62 벚꽃이 피면 우리 그만 헤어져
63 낙하 (with 아이유)
64 아주, 천천히
65

In [14]:
result_df

,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,TOMBOY,(여자)아이들,2022.03.14,"109,821",I NEVER DIE,록/메탈,Look at you 넌 못 감당해 날\nYa took off hook\n기분은 C...
1,GANADARA (Feat. 아이유),박재범,2022.03.11,"87,415",GANADARA,R&B/Soul,길거리를 걷다 보면\n사랑 노래만 흘러나와\n나는 왜 저런 게 낯설까\n난 한국말까...
2,Feel My Rhythm,Red Velvet (레드벨벳),2022.03.21,"80,399",‘The ReVe Festival 2022 - Feel My Rhythm’,댄스,Red Velvet\n\n무도회를 뒤집어\n작은 소란을 또 일으켜\nThis is ...
3,사랑인가 봐,멜로망스,2022.02.18,"49,340",사랑인가 봐 (사내맞선 OST 스페셜 트랙),"발라드, 국내드라마",너와 함께 하고 싶은 일들을\n상상하는 게\n요즘 내 일상이 되고\n너의 즐거워하는...
4,취중고백,김민석 (멜로망스),2021.12.19,"99,275",취중고백,발라드,뭐하고 있었니 늦었지만\n잠시 나올래\n너의 집 골목에 있는\n놀이터에 앉아 있어\...
...,...,...,...,...,...,...,...
95,우연히 봄,"로꼬, 유주 (YUJU)",2015.04.08,"157,771",냄새를 보는 소녀 OST Part.2,"발라드, 랩/힙합, 국내드라마",우연히 내게 오나봐\n봄 향기가 보여\n너도 같이 오나봐\n저 멀리서 니 향기가\n...
96,노래 (The Song),비투비,2022.02.21,"35,519",Be Together,발라드,어느새 흐려진 수많은 날들이\n문득 생각이 나죠\n그때 즐겨 듣던 노랠\n하나씩 꺼...
97,HERO,임영웅,2020.11.04,"71,612",HERO,성인가요/트로트,세상이란 장애물이 너의\n앞길을 가로막을 때 날 봐\n언제나 너의 곁엔 내가 있어\...
98,늦은 후회,조현아 (어반자카파),2022.03.18,"7,143",늦은 후회,발라드,얼마나 한참을 서있었는지\n멀리 너의 모습 보면서\n그 모습 사라질 때까지\n나의 ...


In [15]:
for i in range(100) :
    try :
        result_df['Lyric'][i] = result_df['Lyric'][i].replace('\n',' ')
    except :
        continue

In [16]:
result_df['Lyric']

0     Look at you 넌 못 감당해 날 Ya took off hook 기분은 Cok...
1     길거리를 걷다 보면 사랑 노래만 흘러나와 나는 왜 저런 게 낯설까 난 한국말까지 서...
2     Red Velvet  무도회를 뒤집어 작은 소란을 또 일으켜 This is gonn...
3     너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...
4     뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...
                            ...                        
95    우연히 내게 오나봐 봄 향기가 보여 너도 같이 오나봐 저 멀리서 니 향기가 설레는 ...
96    어느새 흐려진 수많은 날들이 문득 생각이 나죠 그때 즐겨 듣던 노랠 하나씩 꺼내 보...
97    세상이란 장애물이 너의 앞길을 가로막을 때 날 봐 언제나 너의 곁엔 내가 있어 불안...
98    얼마나 한참을 서있었는지 멀리 너의 모습 보면서 그 모습 사라질 때까지 나의 발걸음...
99    내가 이렇게 아픈데 그댄 어떨까요 원래 떠나는 사람이 더 힘든 법인데 아무 말하지 ...
Name: Lyric, Length: 99, dtype: object

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
from afinn import Afinn
from nltk.corpus import stopwords
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

dataset = pd.read_csv("sentiment_analysis.csv",encoding = 'utf-8')
dataset.head()

,number,value,age,sex,keyword,disease,sentiment_big,sentiment_small,person1,system1,person2,system2,person3,system3
0,24070,1,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,지금 난 기분이 너무 좋아.,굉장히 즐거우신 것 같은데요? 좋은 일이 있나요?,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.,그거 정말 기분 좋을 만하네요. 무엇을 할 예정인가요?,그냥 집에서 뒹굴뒹굴하고 넷플릭스 보고 놀 거야.,집에서 노는 것이 최고죠. 생각만 해도 행복하겠어요.
1,27879,1,청년,여성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,NaN,NaN
2,51471,1,중년,여성,"직장, 업무 스트레스",해당없음,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,NaN,NaN
3,7140,1,노년,남성,대인관계,만성질환 유,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,몸이 아픈데 아내가 이혼하자고 하니 혼자 있는 것 같은 기분을 느끼셨군요.,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.,지금의 기분에서 벗어나기 위해 무엇을 하면 좋을까요?,아내에게 졸혼은 어떤지 대화를 나누어 봐야겠어. 졸혼을 한다면 가끔씩이라도 도움을 ...,아내와의 대화로 기분이 나아졌으면 좋겠어요.
4,39663,1,노년,남성,재정,만성질환 유,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,NaN,NaN


In [3]:
sentimential_list = {'행복' : 0, '기쁨': 0, '분노' : 1, '불안' : 1, '당황' : 1, '슬픔' : 2, '상처' : 2}
dataset['label'] = dataset['sentiment_big'].map(sentimential_list)
dataset.head()

,number,value,age,sex,keyword,disease,sentiment_big,sentiment_small,person1,system1,person2,system2,person3,system3,label
0,24070,1,청년,여성,"진로, 취업, 직장",해당없음,기쁨,신이 난,지금 난 기분이 너무 좋아.,굉장히 즐거우신 것 같은데요? 좋은 일이 있나요?,오늘부터 연휴가 시작됐거든. 내일 회사 안 간다.,그거 정말 기분 좋을 만하네요. 무엇을 할 예정인가요?,그냥 집에서 뒹굴뒹굴하고 넷플릭스 보고 놀 거야.,집에서 노는 것이 최고죠. 생각만 해도 행복하겠어요.,0
1,27879,1,청년,여성,"연애, 결혼, 출산",해당없음,기쁨,신이 난,나도 조카가 생겨! 너무 기뻐.,정말 좋은 소식이네요. 축하할 특별한 계획이 있을까요?,응. 언니에게 꽃다발을 선물할 거야. 예쁜 딸이면 좋을 것 같아서.,언니에게 꽃을 선물하려고 하시는군요.,NaN,NaN,0
2,51471,1,중년,여성,"직장, 업무 스트레스",해당없음,불안,불안,거래처와의 다음 계약이 무산될까봐 불안해.,많이 걱정되시겠어요. 계약이 무산될 것 같다고 생각하는 이유가 있으신가요?,딱히 이유는 없어. 그냥 나 혼자 불안한 상태야.,계약을 성공적으로 마무리하실 수 있기를 기원해요.,NaN,NaN,1
3,7140,1,노년,남성,대인관계,만성질환 유,슬픔,염세적인,당뇨로 고생 중인데 아내가 이혼하자더군. 아내가 날 버리면 병은 더 악화될 텐데 세...,몸이 아픈데 아내가 이혼하자고 하니 혼자 있는 것 같은 기분을 느끼셨군요.,젊은 시절에 가정에 소홀했지. 그래도 그렇지 이혼을 하자니 너무 허망해.,지금의 기분에서 벗어나기 위해 무엇을 하면 좋을까요?,아내에게 졸혼은 어떤지 대화를 나누어 봐야겠어. 졸혼을 한다면 가끔씩이라도 도움을 ...,아내와의 대화로 기분이 나아졌으면 좋겠어요.,2
4,39663,1,노년,남성,재정,만성질환 유,슬픔,마비된,노년에 재정적으로 이루어 놓은게 없어서 걱정이야.,나이들어 재정적으로 어려워서 걱정되시는군요.,정말이지 막막해.,막막한 현실이지만 분명 해결책을 찾을 수 있을거예요.,NaN,NaN,2


In [4]:
import re
dataset['text'] = dataset['person1'].map(str) + dataset['system1'].map(str) + dataset['person2'].map(str)+ dataset['system2'].map(str) + dataset['person3'].map(str) + dataset['system3'].map(str)
new_data = {"text" : "ㅎㅎ","label": 0}
dataset = dataset.append(new_data, ignore_index = True)

x_data = dataset['text']
y_data = dataset['label'].values

x_data = x_data.apply(lambda x : re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]', " ", x)).values
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, stratify = y_data, test_size = 0.2, random_state = 42)


from konlpy.tag import Okt

okt = Okt()
def tw_tokenizer(text):
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [5]:
# label 감정별 데이터 분포

dataset['label'].value_counts()

1    36194
2    26213
0    12450
Name: label, dtype: int64

In [6]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score
import time

In [18]:
from lightgbm import LGBMClassifier

def light_gbm_train(x_train_okt, x_test_okt, y_train, y_test):
    print("LGBM Classifier\n")
    start_time = time.time()
    lgbm_clf = LGBMClassifier(n_estimators = 400, n_jobs=-1, reg_alpha = 0.9, reg_lambda = 0.2)
    lgbm_clf.fit(x_train_okt, y_train)
    preds = lgbm_clf.predict(x_test_okt)
    confusion = confusion_matrix(y_test, preds)
    accuracy = accuracy_score(y_test, preds)
    
    precision = precision_score(y_test, preds, average='weighted')
    recall = recall_score(y_test, preds, average='weighted')
    f1 = f1_score(y_test, preds, average='weighted')
    
    print(confusion)
    print('정확도: {0:.4f}\n정밀도: {1:.4f}\n재현율: {2:.4f}\nf1: {3:.4f}'.format(accuracy,precision,recall,f1))
    print(f'fit time : {time.time() - start_time}')

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(ngram_range = (1, 2), min_df = 3, max_df = 0.9, tokenizer = okt.morphs, token_pattern = None)
tfidf.fit(x_train)
x_train_okt = tfidf.transform(x_train)
x_test_okt = tfidf.transform(x_test)

In [8]:
from lightgbm import LGBMClassifier

lgbm_clf = LGBMClassifier(n_estimators = 400, n_jobs=-1, reg_alpha = 0.9, reg_lambda = 0.2)
lgbm_clf.fit(x_train_okt, y_train)

LGBMClassifier(n_estimators=400, reg_alpha=0.9, reg_lambda=0.2)

In [17]:
lyric_sentiment_list = []

In [18]:
for i in range(0,result_df.shape[0]):
    try :
        if result_df['Lyric'][i] != [] :
            lyric_sentiment = list(lgbm_clf.predict(tfidf.transform([result_df['Lyric'][i]])))
            lyric_sentiment_list.append(lyric_sentiment[0])
    except:
        continue

In [19]:
lyric_sentiment_list

[2,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 0,
 2,
 1,
 1,
 2,
 2,
 1,
 0,
 2,
 0,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 0,
 0,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 0,
 2]

In [20]:
lyric_sentiment_df = pd.DataFrame(lyric_sentiment_list)
lyric_sentiment_df.columns = ["label"]
lyric_sentiment_df

,label
0,2
1,1
2,1
3,1
4,2
...,...
93,2
94,1
95,1
96,0


In [21]:
result_df

,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric
0,TOMBOY,(여자)아이들,2022.03.14,"109,821",I NEVER DIE,록/메탈,Look at you 넌 못 감당해 날 Ya took off hook 기분은 Cok...
1,GANADARA (Feat. 아이유),박재범,2022.03.11,"87,415",GANADARA,R&B/Soul,길거리를 걷다 보면 사랑 노래만 흘러나와 나는 왜 저런 게 낯설까 난 한국말까지 서...
2,Feel My Rhythm,Red Velvet (레드벨벳),2022.03.21,"80,399",‘The ReVe Festival 2022 - Feel My Rhythm’,댄스,Red Velvet 무도회를 뒤집어 작은 소란을 또 일으켜 This is gonn...
3,사랑인가 봐,멜로망스,2022.02.18,"49,340",사랑인가 봐 (사내맞선 OST 스페셜 트랙),"발라드, 국내드라마",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...
4,취중고백,김민석 (멜로망스),2021.12.19,"99,275",취중고백,발라드,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...
...,...,...,...,...,...,...,...
95,우연히 봄,"로꼬, 유주 (YUJU)",2015.04.08,"157,771",냄새를 보는 소녀 OST Part.2,"발라드, 랩/힙합, 국내드라마",우연히 내게 오나봐 봄 향기가 보여 너도 같이 오나봐 저 멀리서 니 향기가 설레는 ...
96,노래 (The Song),비투비,2022.02.21,"35,519",Be Together,발라드,어느새 흐려진 수많은 날들이 문득 생각이 나죠 그때 즐겨 듣던 노랠 하나씩 꺼내 보...
97,HERO,임영웅,2020.11.04,"71,612",HERO,성인가요/트로트,세상이란 장애물이 너의 앞길을 가로막을 때 날 봐 언제나 너의 곁엔 내가 있어 불안...
98,늦은 후회,조현아 (어반자카파),2022.03.18,"7,143",늦은 후회,발라드,얼마나 한참을 서있었는지 멀리 너의 모습 보면서 그 모습 사라질 때까지 나의 발걸음...


## 0 : 기쁨 행복
## 1 : 분노 
## 2: 불안 당황
## 3: 슬픔 상처

In [22]:
Liryc_df = pd.merge(result_df,lyric_sentiment_df,left_index= True, right_index=True )
Liryc_df

,title,artist,Album_date,LIKE,Album_name,Album_Genre,Lyric,label
0,TOMBOY,(여자)아이들,2022.03.14,"109,821",I NEVER DIE,록/메탈,Look at you 넌 못 감당해 날 Ya took off hook 기분은 Cok...,2
1,GANADARA (Feat. 아이유),박재범,2022.03.11,"87,415",GANADARA,R&B/Soul,길거리를 걷다 보면 사랑 노래만 흘러나와 나는 왜 저런 게 낯설까 난 한국말까지 서...,1
2,Feel My Rhythm,Red Velvet (레드벨벳),2022.03.21,"80,399",‘The ReVe Festival 2022 - Feel My Rhythm’,댄스,Red Velvet 무도회를 뒤집어 작은 소란을 또 일으켜 This is gonn...,1
3,사랑인가 봐,멜로망스,2022.02.18,"49,340",사랑인가 봐 (사내맞선 OST 스페셜 트랙),"발라드, 국내드라마",너와 함께 하고 싶은 일들을 상상하는 게 요즘 내 일상이 되고 너의 즐거워하는 모습...,1
4,취중고백,김민석 (멜로망스),2021.12.19,"99,275",취중고백,발라드,뭐하고 있었니 늦었지만 잠시 나올래 너의 집 골목에 있는 놀이터에 앉아 있어 친구들...,2
...,...,...,...,...,...,...,...,...
93,흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,장범준,2019.08.23,"299,895",멜로가 체질 OST Part 3,"록/메탈, 국내드라마",흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야 스쳐지나간건가 뒤돌아보지만 그냥 사람들...,2
94,별빛 같은 나의 사랑아,임영웅,2021.03.09,"71,255",별빛 같은 나의 사랑아,성인가요/트로트,당신이 얼마나 내게 소중한 사람인지 세월이 흐르고 보니 이제 알 것 같아요 당신이 ...,1
95,우연히 봄,"로꼬, 유주 (YUJU)",2015.04.08,"157,771",냄새를 보는 소녀 OST Part.2,"발라드, 랩/힙합, 국내드라마",우연히 내게 오나봐 봄 향기가 보여 너도 같이 오나봐 저 멀리서 니 향기가 설레는 ...,1
96,노래 (The Song),비투비,2022.02.21,"35,519",Be Together,발라드,어느새 흐려진 수많은 날들이 문득 생각이 나죠 그때 즐겨 듣던 노랠 하나씩 꺼내 보...,0


In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup
import time 
import re
import json 
import pandas as pd
from datetime import datetime
import string

In [25]:
## id 와 password 입력

user_id=""
user_passwd=""

In [26]:
login_option="instagram" # facebook or instagram 
driver_path="chromedriver.exe" 
instagram_id_name="username" 
instagram_pw_name="password"
instagram_login_btn=".sqdOP.L3NKy.y3zKF "
instagram_information_save_name = ".sqdOP.yWX7d.y3zKF"
instagram_alarm_name = ".aOOlW.HoLwm"
is_information_save = False
is_alarm_setting = False

driver = webdriver.Chrome("./chromedriver")

In [27]:
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)

if login_option == "instagram": 
    time.sleep(2)
    try: 
        instagram_id_form = driver.find_element_by_name(instagram_id_name) 
        instagram_id_form.send_keys(user_id) 
        
        instagram_pw_form = driver.find_element_by_name(instagram_pw_name) 
        instagram_pw_form.send_keys(user_passwd) 
        
        login_ok_button = driver.find_element_by_css_selector(instagram_login_btn) 
        login_ok_button.click() 
        is_login_success = True
        
        time.sleep(3)
        if is_information_save == False and is_alarm_setting == False:
            information_save_btn = driver.find_element_by_css_selector(instagram_information_save_name)
            information_save_btn.click()
            is_information_save = True
            
            alarm_setting_btn = driver.find_element_by_css_selector(instagram_alarm_name)
            alarm_setting_btn.click()
            is_alarm_setting = True
            
    except: 
        print("instagram login fail") 
        is_login_success = False

In [28]:
## 위 내용까지 로그인

my_account_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/span"
my_account_button = driver.find_element_by_xpath(my_account_xpath)
my_account_button.click()

my_profile_full_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/div[2]/div[2]/div[2]/a[1]/div"
my_profile_button = driver.find_element_by_xpath(my_profile_full_xpath)
my_profile_button.click()

time.sleep(1.5)
# 게시물 개수
post_count = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/header/section/ul/li[1]/div/span").text
post_count = int(post_count)

print("게시물 수 : ",post_count)

게시물 수 :  13


In [29]:
insta_df = pd.DataFrame()
new_data = {"date" : "날짜", "place":"장소","content":"본문내용","hashtag":"해시태그","comment":"댓글내용"}
insta_df = insta_df.append(new_data,ignore_index=True)
insta_df.head()

,date,place,content,hashtag,comment
0,날짜,장소,본문내용,해시태그,댓글내용


In [30]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

for i in range(post_count) :
    # 날짜 데이터 크롤링
    time.sleep(2)
    date = driver.find_element_by_css_selector('time._1o9PC')
    time.sleep(2)
    date_info = pd.to_datetime(date.get_attribute('datetime')).normalize()
    date = date_info.strftime("%Y년 %m월 %d일")
    print(date)
    #print(date_info)
    #print(date_info.year+"년")
    #date = str(date_info.year) + "년 " + str(date_info.month) + "월 " + str(date_info.day) +"일"

    # 장소 크롤링
    time.sleep(0.5)
    try : 
        place = driver.find_element_by_css_selector("body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.UE9AK > div > header > div.o-MQd.z8cbW > div.M30cS > div > div.JF9hh > div > a").text
    except : 
        place = None

    try : 
        post_content = driver.find_element_by_xpath("/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/div/li/div/div/div[2]/div[1]/span").text    
    except :
        post_content = None
    post_content_list =[]
    post_content_list.append(post_content)
   
    # 본문 속 해시태그 정제
    if post_content != None :
        post_content_tag= []
        post_content = re.findall("#[A-Za-z0-9가-힣]+", post_content)
        content = ''.join(post_content).replace("#"," ") # "# 제거"
        content_split = content.split()
        
        for tag in content_split :
            post_content_tag.append(tag)
    
    # 댓글 크롤링
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    comment_count = 0

    # 댓글 있는지 확인
    try  :
        ul_list = soup.find_all('ul',  'Mr508')
        comment_count = len(ul_list)
    except :
        # 댓글 없으면
        comment_count = 0
    else : 
        # 댓글 개수 적용하여 대댓글 열기버튼 xpath 추출
        for i in range(1,comment_count+1) :
            comment_list = []
            button_xpath = "/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[" + str(i) + "]/li/ul/li/div/button"
            try : 
                # 대댓글 열기버튼 있으면 클릭
                driver.find_element_by_xpath(button_xpath).click()
                time.sleep(1)
                button_check = driver.find_element_by_class_name("EizgU").text
                while "답글 숨기기" not in button_check : 
                    driver.find_element_by_xpath(button_xpath).click()
                    button_check = driver.find_element_by_class_name("EizgU").text
                    
            except :
                button_xpath = None
            
        # 대댓글이 모두 열려있는 상태에서 html parser        
        # 추가 댓글이 모두 열려있는 상태에서 html parser        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ul_list = soup.find_all("ul","Mr508")
        for d in ul_list :
            user_id = d.find("h3","_6lAjh").find("a").get_text()

            comment = d.find("div","C4VMK").find("div","MOdxS").get_text()

            comment_list.append(user_id+":"+comment)
            try :
                re_ul_list = d.find_all("ul","TCSYW")
                for dd in re_ul_list :
                    #print("[ 답글시작 ]")
                    try : 
                        re_div_list = dd.find_all("div","ZyFrc")
                        for ddd in re_div_list :
                            #print("[답글의 답글 시작]")
                            comment_user_id = ddd.find("h3","_6lAjh").find("a").get_text()
                            
                            comment_comment = ddd.find("div","C4VMK").find("div","MOdxS").get_text()

                            comment_list.append(comment_user_id+":"+comment_comment)
                    except : 
                        re_div_list =""
            except : 
                re_ul_list = ""

    new_data = {"date" : date, "place":place, "content" : post_content_list,"hashtag" : post_content_tag,"comment" : comment_list}
    insta_df = insta_df.append(new_data,ignore_index=True)
    # 다음 버튼
    try :
        next_post_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.Z2Inc._7c9RR > div > div.l8mY4.feth3 > button")
        next_post_button.click()
        time.sleep(0.5)
    except :
        # 닫기 버튼    
        exit_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.NOTWr > button > div > svg")
        exit_button.click()
        break

2022년 03월 27일
2022년 02월 24일
2021년 08월 29일
2021년 07월 11일
2021년 07월 11일
2021년 05월 06일
2021년 03월 15일
2020년 11월 11일
2020년 09월 02일
2020년 08월 12일
2020년 07월 24일
2020년 05월 28일
2020년 05월 24일


In [31]:
insta_df

,date,place,content,hashtag,comment
0,날짜,장소,본문내용,해시태그,댓글내용
1,2022년 03월 27일,서울숲,[#하루필름],[하루필름],"[uoah.ns_:귀여워, go_ne_jin:@uoah.ns_ ❤️ 언니 올해는 5..."
2,2022년 02월 24일,어마무시 티라미수,[2022❤️],[],"[zzae_v:다 좋은데 신발은…?, go_ne_jin:@zzae_v 벗어 던져버렸..."
3,2021년 08월 29일,낙산사,[개강 전 finale\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n혜진...,[],"[mins_lucule:아 낙산사 뭘좀 아네 이 자슥? ̊̈ 😍, suuout:👶,..."
4,2021년 07월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[나도 기본캠📸플래쉬 감성 ?\n.\n.\n.\n.\n.\n.\n.\n.\n포둉포둉...,[],"[perarduaadastra__:어디가 포동포동이야 전혀 😮, go_ne_jin:..."
5,2021년 07월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[글램핑\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n인 척 ….],[],"[i_love_wooriwoori:이뿌다👏😍, go_ne_jin:@i_love_wo..."
6,2021년 05월 06일,익선동,[간만에 🙈\n.\n.\n.\n.\n.\n.\n.\n.\n.\n머리끈 너무 거슬리는데],[],"[i_love_wooriwoori:😍 이뿌다, go_ne_jin:@i_love_wo..."
7,2021년 03월 15일,Bar:eve - 바이브,[이 포즈 이제 컨셉✨],[],"[__j.euniii__:여긴 어댜 맛있어?, go_ne_jin:@_jungeun...."
8,2020년 11월 11일,롯데월드 (lotteworld),[늙어서 많이 못놀게따🙈\n.\n.\n.\n.\n.\n.\n...,[],"[uoah.ns_:왜 나 안데려갔어 -3-, go_ne_jin:@tn_ns99 언니..."
9,2020년 09월 02일,모이핀,[안 올수가 없는 카페야\n.\n.\n.\n.\n.\n.\n...,[],"[zzae_v:여기가 모이핀이군, go_ne_jin:@zzae_v 누가 더 인생샷인..."


In [32]:
comment_re_list = ['댓글내용 정제']   

In [33]:
for k in range(1,insta_df['comment'].shape[0]) :
    comm = " "
    strstr=" "
    for i in range(len(insta_df['comment'][k])) :
        comm = insta_df['comment'][k][i].split(":")
        comm = comm[1]
        str_split = comm.split("@")
        if(len(str_split) != 1) :
            index = str_split[1].find(" ")
            str_split = str_split[1]
            str_split = str_split[index:]
            comm = str_split
        if(comm.find(" ") == 0) :
            comm = comm[1:]
        comm = comm.replace("#"," ")
        strstr += comm
    comment_re_list.append(strstr)

In [34]:
comment_re_df = pd.DataFrame(comment_re_list)
comment_re_df.columns = ["comment_re"]
comment_re_df

,comment_re
0,댓글내용 정제
1,귀여워❤️ 언니 올해는 5월이나 종강할때 꼭 시간내봐요 ㅎㅎ선배 넘 넘 귀엽다!!...
2,다 좋은데 신발은…?벗어 던져버렸어ㅋㅋㅋㅋㅋㅋㅋㅋㅋ너두가라 경주 이뽀다ㅋㅋㅋㅋㅋㅋ...
3,아 낙산사 뭘좀 아네 이 자슥? ̊̈ 😍👶뻬이뻬 K대면에서 만나…혜진 ~ 귤 까조...
4,어디가 포동포동이야 전혀 😮언니 그 photoshop이라구 ㅎㅎㅎㅎ
5,이뿌다👏😍쌤 여름방학때 꼭한번 들릴려고했는데… 단계가 심각해져서 큰일이에요😂코로나...
6,😍 이뿌다방학 때 한번 들릴게요🤙 쌤 우리 엄청컸던데요 ㅎㅎ 시간 너무빨라요ㅠㅜ ...
7,여긴 어댜 맛있어?닭볶음탕이 맛있었오🙌오!!! 그럼 너네 학교 놀러가면 요거 먹자...
8,왜 나 안데려갔어 -3-언니 ㅎㅎ 코딩하고 싶죠 ㅎㅎ🙈자꾸 이러면 강릉쫓아가여지긋...
9,여기가 모이핀이군누가 더 인생샷인지 대결하겠어 ㅎㅎ쫌 예뿌다? 옷이ㅋㅋㅋㅋㅋㅋㅋㅋ...


In [35]:
insta_df = pd.merge(insta_df,comment_re_df,left_index= True, right_index=True )
insta_df

,date,place,content,hashtag,comment,comment_re
0,날짜,장소,본문내용,해시태그,댓글내용,댓글내용 정제
1,2022년 03월 27일,서울숲,[#하루필름],[하루필름],"[uoah.ns_:귀여워, go_ne_jin:@uoah.ns_ ❤️ 언니 올해는 5...",귀여워❤️ 언니 올해는 5월이나 종강할때 꼭 시간내봐요 ㅎㅎ선배 넘 넘 귀엽다!!...
2,2022년 02월 24일,어마무시 티라미수,[2022❤️],[],"[zzae_v:다 좋은데 신발은…?, go_ne_jin:@zzae_v 벗어 던져버렸...",다 좋은데 신발은…?벗어 던져버렸어ㅋㅋㅋㅋㅋㅋㅋㅋㅋ너두가라 경주 이뽀다ㅋㅋㅋㅋㅋㅋ...
3,2021년 08월 29일,낙산사,[개강 전 finale\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n혜진...,[],"[mins_lucule:아 낙산사 뭘좀 아네 이 자슥? ̊̈ 😍, suuout:👶,...",아 낙산사 뭘좀 아네 이 자슥? ̊̈ 😍👶뻬이뻬 K대면에서 만나…혜진 ~ 귤 까조...
4,2021년 07월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[나도 기본캠📸플래쉬 감성 ?\n.\n.\n.\n.\n.\n.\n.\n.\n포둉포둉...,[],"[perarduaadastra__:어디가 포동포동이야 전혀 😮, go_ne_jin:...",어디가 포동포동이야 전혀 😮언니 그 photoshop이라구 ㅎㅎㅎㅎ
5,2021년 07월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[글램핑\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n인 척 ….],[],"[i_love_wooriwoori:이뿌다👏😍, go_ne_jin:@i_love_wo...",이뿌다👏😍쌤 여름방학때 꼭한번 들릴려고했는데… 단계가 심각해져서 큰일이에요😂코로나...
6,2021년 05월 06일,익선동,[간만에 🙈\n.\n.\n.\n.\n.\n.\n.\n.\n.\n머리끈 너무 거슬리는데],[],"[i_love_wooriwoori:😍 이뿌다, go_ne_jin:@i_love_wo...",😍 이뿌다방학 때 한번 들릴게요🤙 쌤 우리 엄청컸던데요 ㅎㅎ 시간 너무빨라요ㅠㅜ ...
7,2021년 03월 15일,Bar:eve - 바이브,[이 포즈 이제 컨셉✨],[],"[__j.euniii__:여긴 어댜 맛있어?, go_ne_jin:@_jungeun....",여긴 어댜 맛있어?닭볶음탕이 맛있었오🙌오!!! 그럼 너네 학교 놀러가면 요거 먹자...
8,2020년 11월 11일,롯데월드 (lotteworld),[늙어서 많이 못놀게따🙈\n.\n.\n.\n.\n.\n.\n...,[],"[uoah.ns_:왜 나 안데려갔어 -3-, go_ne_jin:@tn_ns99 언니...",왜 나 안데려갔어 -3-언니 ㅎㅎ 코딩하고 싶죠 ㅎㅎ🙈자꾸 이러면 강릉쫓아가여지긋...
9,2020년 09월 02일,모이핀,[안 올수가 없는 카페야\n.\n.\n.\n.\n.\n.\n...,[],"[zzae_v:여기가 모이핀이군, go_ne_jin:@zzae_v 누가 더 인생샷인...",여기가 모이핀이군누가 더 인생샷인지 대결하겠어 ㅎㅎ쫌 예뿌다? 옷이ㅋㅋㅋㅋㅋㅋㅋㅋ...


In [36]:
comment_sentiment_list = ['감정분석결과']

In [37]:
for i in range(1,insta_df.shape[0]):
    if insta_df['comment_re'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['comment'][i])))
    if insta_df['content'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['content'][i])))
    if insta_df['hashtag'][i] != [] :
        comment_sentiment=list(lgbm_clf.predict(tfidf.transform(insta_df['hashtag'][i])))
    comment_sentiment_list.append(max(comment_sentiment,key = comment_sentiment.count))
comment_sentiment_list

['감정분석결과', 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1]

In [38]:
comment_sentiment_df = pd.DataFrame(comment_sentiment_list)
comment_sentiment_df.columns = ["label"]
comment_sentiment_df

,label
0,감정분석결과
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,0


## 0 : 기쁨 행복
## 1 : 분노 
## 2: 불안 당황
## 3: 슬픔 상처

In [39]:
insta_df = pd.merge(insta_df,comment_sentiment_df,left_index= True, right_index=True )
insta_df

,date,place,content,hashtag,comment,comment_re,label
0,날짜,장소,본문내용,해시태그,댓글내용,댓글내용 정제,감정분석결과
1,2022년 03월 27일,서울숲,[#하루필름],[하루필름],"[uoah.ns_:귀여워, go_ne_jin:@uoah.ns_ ❤️ 언니 올해는 5...",귀여워❤️ 언니 올해는 5월이나 종강할때 꼭 시간내봐요 ㅎㅎ선배 넘 넘 귀엽다!!...,1
2,2022년 02월 24일,어마무시 티라미수,[2022❤️],[],"[zzae_v:다 좋은데 신발은…?, go_ne_jin:@zzae_v 벗어 던져버렸...",다 좋은데 신발은…?벗어 던져버렸어ㅋㅋㅋㅋㅋㅋㅋㅋㅋ너두가라 경주 이뽀다ㅋㅋㅋㅋㅋㅋ...,1
3,2021년 08월 29일,낙산사,[개강 전 finale\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n혜진...,[],"[mins_lucule:아 낙산사 뭘좀 아네 이 자슥? ̊̈ 😍, suuout:👶,...",아 낙산사 뭘좀 아네 이 자슥? ̊̈ 😍👶뻬이뻬 K대면에서 만나…혜진 ~ 귤 까조...,1
4,2021년 07월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[나도 기본캠📸플래쉬 감성 ?\n.\n.\n.\n.\n.\n.\n.\n.\n포둉포둉...,[],"[perarduaadastra__:어디가 포동포동이야 전혀 😮, go_ne_jin:...",어디가 포동포동이야 전혀 😮언니 그 photoshop이라구 ㅎㅎㅎㅎ,1
5,2021년 07월 11일,하이서울유스호스텔 Hi Seoul Youth Hostel,[글램핑\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n.\n인 척 ….],[],"[i_love_wooriwoori:이뿌다👏😍, go_ne_jin:@i_love_wo...",이뿌다👏😍쌤 여름방학때 꼭한번 들릴려고했는데… 단계가 심각해져서 큰일이에요😂코로나...,1
6,2021년 05월 06일,익선동,[간만에 🙈\n.\n.\n.\n.\n.\n.\n.\n.\n.\n머리끈 너무 거슬리는데],[],"[i_love_wooriwoori:😍 이뿌다, go_ne_jin:@i_love_wo...",😍 이뿌다방학 때 한번 들릴게요🤙 쌤 우리 엄청컸던데요 ㅎㅎ 시간 너무빨라요ㅠㅜ ...,1
7,2021년 03월 15일,Bar:eve - 바이브,[이 포즈 이제 컨셉✨],[],"[__j.euniii__:여긴 어댜 맛있어?, go_ne_jin:@_jungeun....",여긴 어댜 맛있어?닭볶음탕이 맛있었오🙌오!!! 그럼 너네 학교 놀러가면 요거 먹자...,1
8,2020년 11월 11일,롯데월드 (lotteworld),[늙어서 많이 못놀게따🙈\n.\n.\n.\n.\n.\n.\n...,[],"[uoah.ns_:왜 나 안데려갔어 -3-, go_ne_jin:@tn_ns99 언니...",왜 나 안데려갔어 -3-언니 ㅎㅎ 코딩하고 싶죠 ㅎㅎ🙈자꾸 이러면 강릉쫓아가여지긋...,1
9,2020년 09월 02일,모이핀,[안 올수가 없는 카페야\n.\n.\n.\n.\n.\n.\n...,[],"[zzae_v:여기가 모이핀이군, go_ne_jin:@zzae_v 누가 더 인생샷인...",여기가 모이핀이군누가 더 인생샷인지 대결하겠어 ㅎㅎ쫌 예뿌다? 옷이ㅋㅋㅋㅋㅋㅋㅋㅋ...,0
